In [32]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import pandas as pd
from selenium import webdriver
import warnings
import json, folium, warnings
warnings.filterwarnings('ignore')

In [33]:
base_url = 'https://www.ggpolice.go.kr'
main_url = '/main/html.do?menu=HC44'
url = f'{base_url}{main_url}'
result = requests.get(url)
soup = BeautifulSoup(result.text)
driver =webdriver.Chrome('file:///C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver')
driver.get(url)

In [34]:
soup =BeautifulSoup(driver.page_source)
lis =soup.select('.road')
len(lis)

31

##### 3. 하나의 데이터로 원하는 정보 추출

In [35]:
li =lis[0]
sub_url = li.select_one('li > a')['href']
sub_url

'https://www.ggpolice.go.kr/swjb/html.do?menu=HC24'

In [36]:
res =requests.get(sub_url)
title_soup = BeautifulSoup(res.text)

In [37]:
info =title_soup.select_one('.basic_list7')
title_b =info.find_all('li')
title_b

[<li>기관명 : 수원중부경찰서</li>,
 <li>도로명 주소 : 경기도 수원시 장안구 정자천로 199 (우)16335</li>,
 <li>대표전화 : 경찰민원 콜센터 182</li>]

In [38]:
police = info.select('.basic_list7 > li')
for li in police:
    print(li.get_text())

기관명 : 수원중부경찰서
도로명 주소 : 경기도 수원시 장안구 정자천로 199 (우)16335
대표전화 : 경찰민원 콜센터 182


In [39]:
police_dict ={}
for li in police:
    item =li.get_text().split(':')
    police_dict[item[0].strip()] =item[1].strip()
police_dict

{'기관명': '수원중부경찰서',
 '도로명 주소': '경기도 수원시 장안구 정자천로 199 (우)16335',
 '대표전화': '경찰민원 콜센터 182'}

In [40]:
name =', '.join( police_dict['기관명'].split(' , '))
name

'수원중부경찰서'

In [41]:
address =', '.join( police_dict['도로명 주소'].split(' , '))
address

'경기도 수원시 장안구 정자천로 199 (우)16335'

In [42]:
# call =', '.join( police_dict['대표전화'].split(' , '))
# call

In [43]:
lines =[]
for li in lis:
  
    sub_url = li.select_one('li > a')['href']
    res =requests.get(sub_url)
    try:
        title_soup = BeautifulSoup(res.text)
        info =title_soup.select_one('.basic_list7')
        title_b =info.find_all('li')
        police = info.select('.basic_list7 > li')
        for li in police:
                item =li.get_text().split(':')
                
                police_dict[item[0].strip()] =item[1].strip()

        name =( police_dict['기관명'])
        address =( police_dict['도로명 주소']).split('(')[0].strip()
        lines.append({'기관명':name,'주소':address})
    except:
        continue

In [44]:
df=pd.DataFrame(lines)
df.head()

,기관명,주소
0,수원중부경찰서,경기도 수원시 장안구 정자천로 199
1,수원남부경찰서,경기 수원시 영통구 매봉로 52
2,수원서부경찰서,경기 수원시 권선구 서부로 1673
3,안양동안경찰서,경기도 안양시 동안구 동안로159번길 43
4,안양만안경찰서,경기도 안양시 만안구 냉천로 63


In [45]:
df.to_csv('data/경기도 경찰서.csv',index=False)


In [46]:
with open('../04. 지도시각화/data/kakaoapikey.txt') as f:
    kakao_key =f.read()
base_url ='https://dapi.kakao.com/v2/local/search/address.json'
addr ='경기도 수원시 장안구 정자천로 199'                #샘플
url = f'{base_url}?query={quote(addr)}'
header = {'Authorization':f'KakaoAK {kakao_key}'}
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '경기 수원시 장안구 정자동 873-4',
    'b_code': '4111113000',
    'h_code': '4111157300',
    'main_address_no': '873',
    'mountain_yn': 'N',
    'region_1depth_name': '경기',
    'region_2depth_name': '수원시 장안구',
    'region_3depth_h_name': '정자3동',
    'region_3depth_name': '정자동',
    'sub_address_no': '4',
    'x': '126.99631718583',
    'y': '37.2979319496338'},
   'address_name': '경기 수원시 장안구 정자천로 199',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '경기 수원시 장안구 정자천로 199',
    'building_name': '수원중부경찰서',
    'main_building_no': '199',
    'region_1depth_name': '경기',
    'region_2depth_name': '수원시 장안구',
    'region_3depth_name': '정자동',
    'road_name': '정자천로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.99631718583',
    'y': '37.2979319496338',
    'zone_no': '16335'},
   'x': '126.99631718583',
   'y': '37.2979319496338'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [47]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [48]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat,lng

(37.2979319496338, 126.99631718583)

In [50]:
lat_list,lng_list =[],[]
for i in df.index:
    url = f'{base_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [3]:
df['위도'] = lat_list
df['경도'] = lng_list
df

NameError: name 'lat_list' is not defined

In [4]:
map = folium.Map(location=[df.위도.mean(),df.경도.mean()], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i],df.경도[i]],
        popup=folium.Popup(df.주소[i],max_width=200),
        tooltip=df.기관명[i],
        icon=folium.Icon(
        color ='purple',icon_color='white',
        icon='glyphicon glyphicon-pushpin', #glyphicon 아이콘 제공 사이트
        prefix='glyphicon'
    )
    ).add_to(map)
title_html ='<h3 align="center" style ="font-size:20px">수원 소재 주요 대학</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

NameError: name 'folium' is not defined